In [2]:
import os 
import warnings
import sys

import pandas as pd 
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import ElasticNet 

import mlflow 
import mlflow.sklearn

In [3]:
data_path = "data/wine-quality.csv"
data = pd.read_csv(data_path)
data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2485,8.0,0.28,0.32,7.6,0.045,61.0,204.0,0.99543,3.10,0.55,10.1,6
910,6.3,0.33,0.20,5.8,0.040,24.0,144.0,0.99425,3.15,0.63,9.9,5
4308,6.2,0.26,0.29,2.0,0.036,16.0,87.0,0.99081,3.33,0.61,11.8,6
4797,5.7,0.33,0.32,1.4,0.043,28.0,93.0,0.98970,3.31,0.50,12.3,6
4632,6.3,0.30,0.91,8.2,0.034,50.0,199.0,0.99394,3.39,0.49,11.7,6
3330,6.7,0.23,0.33,8.1,0.048,45.0,176.0,0.99472,3.11,0.52,10.1,6
1830,7.7,0.44,0.24,11.2,0.031,41.0,167.0,0.99480,3.12,0.43,11.3,7
4464,6.6,0.26,0.56,15.4,0.053,32.0,141.0,0.99810,3.11,0.49,9.3,5
2768,7.7,0.30,0.34,1.2,0.048,4.0,119.0,0.99084,3.18,0.34,12.1,6
3846,6.7,0.24,0.32,9.0,0.023,20.0,109.0,0.99262,3.34,0.35,12.6,6


In [4]:
remote_server_uri = "http://127.0.0.1:5000/"
mlflow.set_tracking_uri(remote_server_uri)

In [5]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000/'

In [6]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2022/07/05 21:31:45 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='ElasticNet_wine', tags={}>

# WHAT DO WE TRACK ?

* code version : Git commit hash ised for the run (if it was run from an mlflow project) 
* Start & End time : Start and ENd time of the run 
* Source what code run ? 
* parameters : key-value input parameters 
* metrics key-value metrics where the value is numeric (can be updated over the run) 
* Atrifacts : Output files in any format


In [7]:
def eval_metrics(actual, pred) :
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse , mae, r2

In [8]:
def load_data(data_path):
    data = pd.read_csv(data_path)
    
    train, test = train_test_split(data)
    
    train_x = train.drop(["quality"], axis = 1)
    test_x = test.drop(["quality"], axis = 1)
    train_y = train[['quality']]
    test_y = test[['quality']]
    
    return train_x, train_y, test_x, test_y

In [9]:
def train(alpha=0.5, l1_ratio=0.5):
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    
    # read the datta
    
    data_path = "data/wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)
    
    
    # usefull for multiple runs (only doing one run in this notebook )
    
    with mlflow.start_run():
        lr = ElasticNet(alpha = alpha, l1_ratio = l1_ratio, random_state= 42)
        lr.fit(train_x, train_y)
        
        
        # evaluate metrics 
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        
        
        # print()
        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        
        
        # log parameters 
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({'mae': mae, "r2":r2})
        mlflow.log_artifact(data_path)
        print(f"save to {mlflow.get_artifact_uri()}")   
        mlflow.sklearn.log_model(lr, "model")

In [10]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8222428497595401
  MAE: 0.6278761410160693
  R2: 0.12678721972772666
save to ./mlruns/1/4dec4da201394a17bb6adc4f58da0509/artifacts


In [11]:
train(0.2, 0.89)

Elasticnet model (alpha=0.200000, l1_ratio=0.890000):
  RMSE: 0.8022565720493103
  MAE: 0.6218183128184986
  R2: 0.16872170832780642
save to ./mlruns/1/7c1180939cdd4345b1e055f3c2928d6d/artifacts


In [12]:
import numpy as np

In [13]:
for i in [0.001, 0.01, 0.5, 0.1, 1]:
    for j in [0.001, 0.01, 0.5, 0.1, 1]:
        train(i,j)

Elasticnet model (alpha=0.001000, l1_ratio=0.001000):
  RMSE: 0.7503813642195375
  MAE: 0.5834487796546969
  R2: 0.27274962837751293
save to ./mlruns/1/ce5c1bc5fded455e8575549032aa8983/artifacts
Elasticnet model (alpha=0.001000, l1_ratio=0.010000):
  RMSE: 0.7503816415881208
  MAE: 0.5834538315668345
  R2: 0.2727490907403801
save to ./mlruns/1/a1870f5b71f04923826507fa3cddec4b/artifacts
Elasticnet model (alpha=0.001000, l1_ratio=0.500000):
  RMSE: 0.7504340478812798
  MAE: 0.5837497243928481
  R2: 0.2726475054853085
save to ./mlruns/1/adc8c7e9ab7549cb9d51c3dd7b0da569/artifacts
Elasticnet model (alpha=0.001000, l1_ratio=0.100000):
  RMSE: 0.7503826026809187
  MAE: 0.5835025369238694
  R2: 0.2727472278055214
save to ./mlruns/1/e0ee2fd25d2d46e7a2e04e3cce6e7c9f/artifacts
Elasticnet model (alpha=0.001000, l1_ratio=1.000000):
  RMSE: 0.750438944358813
  MAE: 0.5840539021218226
  R2: 0.27263801370711105
save to ./mlruns/1/f2f7af1a7c6a49159fa08bb5fdcd8a6d/artifacts
Elasticnet model (alpha=0.010